In [1]:
!git clone https://github.com/truth771/e2e-sae.git

fatal: destination path 'e2e-sae' already exists and is not an empty directory.


In [2]:
%cd e2e-sae

/content/e2e-sae


In [3]:
!git restore . && git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 432 bytes | 432.00 KiB/s, done.
From https://github.com/truth771/e2e-sae
   ed74c1e..d502458  main       -> origin/main
Updating ed74c1e..d502458
Fast-forward
 sae/training/train.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [4]:
!pip install fairscale

In [5]:
!ls gpt2-pytorch_model.bin || curl --output gpt2-pytorch_model.bin https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin

gpt2-pytorch_model.bin


In [6]:
from sae.training.train import train, SAEParams
from sae.models import get_model

In [19]:
model = train("gpt2", SAEParams(6, 60 * 768, "local"), batch_size=4, n_epochs=1, mse_weight=100)

Token indices sequence length is longer than the specified maximum sequence length for this model (1106 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1086 > 1024). Running this sequence through the model will result in indexing errors
epoch 1:   4%|▍         | 98/2429 [00:31<12:26,  3.12it/s]


In [14]:
from sae.training.datasets import get_openwebtext_dataloaders, OpenWebTextConfig
from sae.eval.active_features import eval_l0
from sae.eval.ce_loss import eval_ce_loss_increase

In [15]:
train_loader, val_loader = get_openwebtext_dataloaders(OpenWebTextConfig(), 4)
eval_l0(model, val_loader, "cuda")

Token indices sequence length is longer than the specified maximum sequence length for this model (1106 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1086 > 1024). Running this sequence through the model will result in indexing errors


52410.66414380322

In [21]:
normal_model = get_model("gpt2", SAEParams)[0].to("cuda")
eval_ce_loss_increase(model, normal_model, val_loader, "cuda")

{'ce_loss_increase': -5.490897824658238,
 'ce_original': 7.63107807588893,
 'ce_sae': 2.1401802450277496}

In [41]:
i = next(iter(val_loader))
model(i["input_ids"].to('cuda'))[0][:, -1, :].mean()

tensor(-43.2869, device='cuda:0')

In [11]:
for sparsity_param in [0.0001, 0.001, 0.01, 0.1, 1.0, 10]:
    for t in "local", "e2e", "e2e + ds":
        train("gpt2", SAEParams(6, 60 * 768, "local"), batch_size=4,
              n_epochs=1, sparsity_weight=sparsity_param, checkpoint_epochs={1})

Token indices sequence length is longer than the specified maximum sequence length for this model (1106 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1086 > 1024). Running this sequence through the model will result in indexing errors
epoch 1:  23%|██▎       | 570/2429 [03:02<09:55,  3.12it/s]


KeyboardInterrupt: 